<a href="https://colab.research.google.com/github/andreidm92/computer_vision_tasks/blob/main/practice/Video_Detection_YOLOv8_Lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Практика: Пайплайн видео + YOLOv8 (День 2)

**Теория: Пайплайн видео + YOLOv8**
Что мы будем строить:

Захват видео с камеры или файла через cv2.VideoCapture.

Построчная обработка каждого кадра.

Передача кадра в модель YOLOv8 для детекции объектов.

Отрисовка предсказаний поверх кадра (bounding boxes, метки объектов).

Показ кадра на экране в реальном времени.

(Дополнительно) Сохранение видео с результатами обработки.

**Мини-архитектура пайплайна**
css
Копировать
Редактировать
[Video Input] → [Чтение кадра] → [YOLOv8 инференс] → [Постпроцессинг] → [Отображение/Запись]



**Что такое YOLOv8**

YOLO (You Only Look Once) — один из самых быстрых и эффективных алгоритмов детекции объектов.

v8 — последняя версия (от Ultralytics) с улучшением скорости и качества.

YOLOv8 поддерживает работу без анкор-боксов, имеет встроенные улучшенные слои и мощные постпроцессоры.

Плюсы YOLOv8:

Очень быстро обрабатывает кадры на CPU и GPU.

Легко интегрируется в пайплайн видео.

Удобный API через библиотеку ultralytics.

In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
import cv2
from ultralytics import YOLO

In [ ]:
# Загрузка видео
video_path = '/content/video_1.mp4'

# Загрузка модели
model = YOLO('yolov8n.pt')

# Открытие видео
cap = cv2.VideoCapture(video_path)

# Подготовка записи обработанного видео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/output_video_1.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
# int(cap.get(3)), int(cap.get(4)))
# cap.get(3) - это значит: Обратись к объекту cap (видео), Получи свойство с номером 3, Свойство 3 = ширина видео в пикселях.

In [ ]:
'''
Что такое кодек в OpenCV
Когда мы записываем видео через cv2.VideoWriter, мы создаём новый видеофайл.

Кодек (от англ. "coder-decoder") — это способ кодирования видео данных в определённый формат:

Как упаковать кадры в файл,

Как сохранить сжатием или без,

Какой стандарт использовать для воспроизведения видео.

Почему без кодека нельзя просто "записать" видео?
Когда ты записываешь много кадров (картинок) подряд:

Без кодека это были бы просто "сырые" пиксели (непонятный набор данных).

Кодек делает так, чтобы из набора кадров получилось реальное видео, которое может воспроизвести любой видеоплеер.

Что именно делает эта строка:
python
Копировать
Редактировать
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
🔵 Она создаёт кодек mp4v (MPEG-4 Video codec):

Это популярный кодек для .mp4 файлов.

Его поддерживают почти все плееры и платформы.

Функция cv2.VideoWriter_fourcc принимает 4 символа (например, 'm', 'p', '4', 'v') и превращает их в код для записи видео.

Потом кодек передаётся сюда:
python
Копировать
Редактировать
out = cv2.VideoWriter('/content/output_video.mp4', fourcc, 20.0, (ширина, высота))
🔵 Что делает VideoWriter:

Создаёт файл /content/output_video.mp4

Сохраняет в него кадры со скоростью 20 fps (кадров в секунду),

Использует кодек mp4v для правильного кодирования и упаковки видео.

'''

In [ ]:
'''
Что такое results в YOLOv8
Когда ты делаешь results = model(frame), модель возвращает список объектов типа ultralytics.engine.results.Results.

Каждый элемент results[i] содержит:

boxes — координаты ограничивающих прямоугольников (bounding boxes).

scores — confidence-оценки (уверенность модели).

classes — классы объектов (например, 0 — человек, 2 — автомобиль).

+ еще: маски, keypoints (если есть).

То есть results[0] это не просто список координат!
Это целый объект с множеством информации.
'''

while cap.isOpened():
    ret, frame = cap.read()
    # ret — это флаг, который показывает, получилось ли считать кадр (True или False)
    if not ret:
        break

    # Детекция объектов
    results = model(frame)
    # Модель возвращает результаты детекции: bounding boxes, классы объектов, confidence scores.

    # Отрисовка результатов
    annotated_frame = results[0].plot()

    # Сохранение кадра в выходной файл
    out.write(annotated_frame)

cap.release()
out.release()

print("Готово! Видео сохранено в /content/output_video_1.mp4")
